<a href="https://colab.research.google.com/github/fhac-ewi/recurrent-neural-network/blob/main/%C3%9Cbung_RNN_L%C3%B6sung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Übung RNN

## In dieser Übung ...
... beschäftigen wir uns mit rekurrenten Neuronen. 

TODO Vervollständigung, Inhalt und Ziele der Übung herausstellen.






## x.1 One Hot Kodierung

TODO Einleitung schreiben.

**Ihre Aufgaben**

(1) TODO (Weitere Aufgaben als weitere Textzelle einfügen, damit man theoretisch direkt die Antwort unter die Aufgabe programmieren kann)

## x.2 Erstellung eines eigenen Datensatzes 

TODO Einleitung schreiben.

**Ihre Aufgaben**

(1) TODO

## x.3 Training eines RNN Modells

TODO Einleitung schreiben.

**Ihre Aufgaben**

(1) TODO

## x.4 Vergleich mit LSTM und GRU

TODO Einleitung schreiben.

**Ihre Aufgaben**

(1) TODO

## x.5 (Optional) Try it yourself

TODO Einleitung schreiben.

**Ihre Aufgaben**

(1) TODO

## Wusstest du? 
Mit dem Wissen aus dieser Vorlesungseinheit/Übung könntest du deine eigene Autovervollständigung programmieren. 

TODO Erklären wie das geht. Nützliche Links einstreuen.